<a href="https://colab.research.google.com/github/ricotta-jpgomes/yt_datatools/blob/main/ytRelated_extractor_v1/getNodes_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get nodes: obtendo os detalhes dos vídeos pesquisados
<br>Agora que temos as ids possíveis dos vídeos recomendados em relação à nossa seed, vamos retomar as requisições para obter os detalhes de cada um, como visualizações e quantidade de comentários, que serão armazenados também em uma planilha, que será a planilha referente aos nós da nossa rede. A primeira etapa dessa extração pode ser encontrada [nesse notebook](https://github.com/ricotta-jpgomes/yt_datatools/blob/main/ytRelated_extractor_v1/getEdges_module.ipynb)

##1. Instalando e importando Dependências

In [ ]:
# cliente google
!pip install --upgrade google-api-python-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.7 MB 7.5 MB/s 
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11


In [ ]:
# dotenv
! pip install python_dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**1.1. Conectando o Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##2. Resgatando funções predefinidas

In [ ]:
import sys # biblioteca utilizada para resgatar funções definidas em arquivos .py separados

In [ ]:
# adicionando caminho para a pasta com os arquivos .py externos
sys.path.insert(0, '/content/drive/MyDrive/Colab Notebooks/youTube/ytRelated_extractor_v1/helpers') 

In [ ]:
from general import get_extraction_date # função para obter a data de extração dos dados
from youtube import get_video_details # função para obter detalhes de um vídeo a partir do seu id

## 3. Recuperando dados de log.txt e fazendo as requisições

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/youTube/ytRelated_extractor_v1/log.txt', 'r') as f:
  video_id = f.readline() # id do meu vídeo seed
  path = f.readline() # caminho do arquivo com as coordenadas de source - target
  
  videos_ids = [video_id] # crio uma lista contendo meu vídeo seed

  # adiciono o id dos vídeos obtidos na primeira etapa da extração à lista já definida 
  with open(path, 'r') as f: 
      reader = csv.DictReader(f)
      for row in reader:
        videos_ids.append(row['target'])

Eu puxo as ids pela coluna 'target' pois é a coluna onde cada id aparece apenas uma vez, ao contrário da coluna 'source', onde as ids se repetem pela quantidade de vídeos relacionados. Além disso, as ids dos vídeos obtidas na última iteração não constam na coluna 'source', uma vez que elas só seriam adicionadas ali caso houvesse uma nova iteração. Pegando as ids pela coluna 'target' nenhuma fica de fora.

**3.1. Obtendo os dados**
<br>
<br>
Agora, para cada id coletada na primeira etapa do processo, vamos buscar via api informações relativas ao vídeo correspondente.

In [ ]:
videos_details = [] # as informações (em dicionário) serão adicionadas nessa lista

for id in videos_ids:
  video_info = get_video_details(id)

  videos_details.append(video_info) 

## 4. Salvando os dados obtidos

In [ ]:
import csv # biblioteca para manipulação de arquivos .csv

Vou começar definindo o nome e o caminho correspondente para o arquivo que irá receber os dados coletados:

In [ ]:
# Definindo caminho para arquivo que receberá os dados das requisições
file_name = input('digite o nome do arquivo: ') + get_extraction_date()[1]
path = f'/content/drive/MyDrive/Datasets/network_files/{file_name}.csv'

digite o nome do arquivo: teste


In [ ]:
# Cabeçalho da tabela final
headers = ['videoId', 'title', 'description', 'publishedAt', 'channelId', 
           'channelTitle', 'views', 'likes', 'comments' ]

# Gerando e gravando meu arquivo .csv
with open(path, 'w') as f:
  writer = csv.DictWriter(f, fieldnames=headers)
  writer.writeheader()

  for row in videos_details:
    writer.writerow(row)